主要概念來源:

https://xstrader.net/15rule/  第三條: 當一個產業的毛利率往上走時，挑每股營收最高的股票

產業毛利率: 整個板塊個別股票的毛利率成長率，取中位數


==========爬蟲部分棄用=======
參考網頁https://leemeng.tw/beautifulsoup-cheat-sheet.html

先用爬的，缺點是沒辦法比較上一季QoQ是否成長。

In [3]:
'''
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

driver = webdriver.Chrome(executable_path='./chromedriver')
driver.implicitly_wait(20)
driver.get(url='https://stock.wespai.com/p/20822')

html_source = driver.page_source
driver.quit()
'''

"\nfrom selenium import webdriver\nfrom bs4 import BeautifulSoup\nimport pandas as pd\nimport numpy as np\n\ndriver = webdriver.Chrome(executable_path='./chromedriver')\ndriver.implicitly_wait(20)\ndriver.get(url='https://stock.wespai.com/p/20822')\n\nhtml_source = driver.page_source\ndriver.quit()\n"

In [2]:
'''soup = BeautifulSoup(html_source, 'lxml')'''

NameError: name 'html_source' is not defined

In [4]:
'''table = soup.select('table')[0]'''

In [5]:
'''table = soup.find('table', {'class': 'display'})'''

In [6]:
#爬出欄位名稱
'''
columns = [th.text.replace('\n', '') for th in table.find('tr').find_all('th')]
columns
'''

['代號', '公司', '股價', '產業類型', '(年)營業毛利率(%)', '董監持股(%)']

In [7]:
'''
trs = table.find_all('tr')[1:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
rows[:5]
'''

[['9962', '有益', '9.43', '不鏽鋼', '3.72', '15.68'],
 ['9960', '邁達康', '28.85', '球具', '33.87', '20.66'],
 ['9958', '世紀鋼', '88', '鋼結構', '15.41', '22.33'],
 ['9955', '佳龍', '19.3', '環保工程', '3.72', '38.14'],
 ['9951', '皇田', '75', '汽車零組件', '29.1', '29.55']]

In [8]:
'''
df = pd.DataFrame(data=rows, columns=columns)
df.head()
'''

,代號,公司,股價,產業類型,(年)營業毛利率(%),董監持股(%)
0,9962,有益,9.43,不鏽鋼,3.72,15.68
1,9960,邁達康,28.85,球具,33.87,20.66
2,9958,世紀鋼,88,鋼結構,15.41,22.33
3,9955,佳龍,19.3,環保工程,3.72,38.14
4,9951,皇田,75,汽車零組件,29.1,29.55


In [9]:
'''df['(年)營業毛利率(%)']=pd.DataFrame(df['(年)營業毛利率(%)'],dtype=np.float)'''

In [10]:
'''df1=df.groupby("產業類型").agg({'(年)營業毛利率(%)':'median'})'''

In [12]:
#選出毛利率最高的行業
'''df1.sort_values('(年)營業毛利率(%)',ascending=False).head(20)'''

,(年)營業毛利率(%)
產業類型,
矽智財廠,100.000
IP設計公司,100.000
處理器核心,99.830
手工具機,99.470
團購,94.170
Internet相關,89.970
證券,86.715
西藥製造業,86.540
醫材開發廠商,86.500


抓CM的資料來運算

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tushare as ts
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats
import math
import scipy.stats as scs
import statsmodels.api as sm
from pylab import mpl, plt
import datetime
from datetime import datetime
data=pd.read_excel('file.xlsx',skiprows = 4, sheet_name = '工作表1'  )
data.head()

,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
0,1101,台泥,水泥工業,29.47,26.30,35.71,-2.67,10178622.0,0.120532,-0.107567
1,1102,亞泥,水泥工業,28.07,20.53,32.27,-12.83,6798878.0,0.367267,-0.268614
2,1103,嘉泥,水泥工業,7.37,-3.93,-0.62,17.18,173129.0,-2.875318,-1.533243
3,1104,環泥,水泥工業,15.20,13.44,16.84,13.66,477040.0,0.130952,-0.115789
4,1108,幸福,水泥工業,10.52,16.55,18.75,23.73,384035.0,-0.364350,0.573194


In [29]:
Columns=list(data.columns)

In [30]:
Columns

['股票代號',
 '股票名稱',
 '2020產業名稱',
 '2019Q4毛利率(%)',
 '2020Q1毛利率(%)',
 '2020Q2毛利率(%)',
 '202007近三月合併營收年成長(%)',
 '202007單月合併營收(千)',
 '自訂.季毛利率成長率',
 '自訂.前兩季的毛利率QoQ成長率']

In [31]:
data.columns[5]

'2020Q2毛利率(%)'

In [32]:
#做變數處理，由於CM的資料，是選最新一期，因此單月合併營收這一欄位的名稱會變，裡面含有月份。
columns_list=list(data.columns)
c_1 = [i for i in columns_list if '單月合併營收(千)' in i]
one_month_income=data.columns[columns_list.index(c_1[0])]
#變數處理，近三月合併營收年成長欄位
c_2 = [i for i in columns_list if '近三月合併營收年成長' in i]
three_months_income=data.columns[columns_list.index(c_2[0])]   
#變數處理，毛利率欄位
profit_rate=data.columns[5]


In [33]:
data['2020Q1毛利率(%)'].median()

21.26

In [34]:
#取出全部股票的毛利率中位數
profit_filter=data.agg({profit_rate:'median'})[0]
profit_filter

22.5

In [35]:
#算出產業的毛利率成長率並做排序
df2=data.groupby("2020產業名稱").agg({'自訂.前兩季的毛利率QoQ成長率':'median'})
print(df2.sort_values('自訂.前兩季的毛利率QoQ成長率',ascending=False).head(10))

df_good_profit=df2.sort_values('自訂.前兩季的毛利率QoQ成長率',ascending=False).head(10)

          自訂.前兩季的毛利率QoQ成長率
2020產業名稱                  
農業科技              0.110870
造紙工業              0.081901
塑膠工業              0.080821
電子商務              0.045480
油電燃氣              0.039179
電子–電子通路           0.029505
建材營建              0.008623
橡膠工業              0.004477
生技醫療              0.000000
金融保險             -0.004742


In [36]:
list(df_good_profit.index)

['農業科技',
 '造紙工業',
 '塑膠工業',
 '電子商務',
 '油電燃氣',
 '電子–電子通路',
 '建材營建',
 '橡膠工業',
 '生技醫療',
 '金融保險']

In [37]:
df_good_profit.index.values

array(['農業科技', '造紙工業', '塑膠工業', '電子商務', '油電燃氣', '電子–電子通路', '建材營建', '橡膠工業',
       '生技醫療', '金融保險'], dtype=object)

In [38]:
#選出單月合併營收最高者
df3=data.groupby(['2020產業名稱'])[one_month_income].transform(max) == data[one_month_income]
data[df3]

,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
0,1101,台泥,水泥工業,29.47,26.30,35.71,-2.67,10178622.0,0.120532,-0.107567
12,1216,統一,食品工業,32.20,34.28,35.13,-1.53,40509558.0,-0.060677,0.064596
26,1240,茂生農經,農業科技,12.72,14.68,16.76,22.86,180860.0,-0.133515,0.154088
33,1303,南亞,塑膠工業,11.40,11.48,12.63,-13.14,21752364.0,-0.006969,0.007018
58,1402,遠東新,紡織纖維,16.28,18.19,19.64,-25.78,15997860.0,-0.105003,0.117322
108,1504,東元,電機機械,20.61,24.83,26.71,-8.00,3940025.0,-0.169956,0.204755
160,1605,華新,電器電纜,11.67,6.27,16.21,-24.42,8900885.0,0.861244,-0.462725
181,1717,長興,化學工業,20.72,21.26,23.77,-12.74,3203507.0,-0.025400,0.026062
197,1736,喬山,生技醫療,48.60,47.13,46.76,9.96,2172534.0,0.031190,-0.030247
216,1802,台玻,玻璃陶瓷,10.75,5.66,9.51,-4.76,3692449.0,0.899293,-0.473488


In [39]:
#選出單月合併營收最高的前3檔股票
df4=data.sort_values(one_month_income, ascending=False)
df5=pd.DataFrame(columns=Columns)
for i in list(df_good_profit.index):
    df5=pd.concat([df5,df4[data["2020產業名稱"]==i].head(3)],axis=0)
df5

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
26,1240,茂生農經,農業科技,12.72,14.68,16.76,22.86,180860.0,-0.133515,0.154088
1428,6508,惠光,農業科技,23.97,33.57,30.42,-17.50,156184.0,-0.285970,0.400501
1459,6578,達邦蛋白,農業科技,14.03,12.25,17.58,-15.62,57797.0,0.145306,-0.126871
229,1907,永豐餘,造紙工業,17.89,19.73,18.94,-10.45,6012155.0,-0.093259,0.102851
230,1909,榮成,造紙工業,16.35,15.71,12.35,-8.16,3862177.0,0.040738,-0.039144
226,1904,正隆,造紙工業,25.02,28.85,25.65,-0.96,3364103.0,-0.132756,0.153078
33,1303,南亞,塑膠工業,11.40,11.48,12.63,-13.14,21752364.0,-0.006969,0.007018
50,1326,台化,塑膠工業,6.23,2.63,10.62,-27.46,20451507.0,1.368821,-0.577849
32,1301,台塑,塑膠工業,11.52,12.62,12.29,-19.91,15716282.0,-0.087163,0.095486
1525,8044,網家,電子商務,10.80,11.91,11.62,16.79,3447881.0,-0.093199,0.102778


In [40]:
bool_filter = data[df3]['2020產業名稱'].isin(list(df_good_profit.index))
result1=data[df3][bool_filter]
result1

,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
26,1240,茂生農經,農業科技,12.72,14.68,16.76,22.86,180860.0,-0.133515,0.154088
33,1303,南亞,塑膠工業,11.40,11.48,12.63,-13.14,21752364.0,-0.006969,0.007018
197,1736,喬山,生技醫療,48.60,47.13,46.76,9.96,2172534.0,0.031190,-0.030247
229,1907,永豐餘,造紙工業,17.89,19.73,18.94,-10.45,6012155.0,-0.093259,0.102851
272,2105,正新,橡膠工業,21.24,18.10,23.25,-13.23,8988799.0,0.173481,-0.147834
533,2882,國泰金,金融保險,19.19,27.15,NaN,28.66,60755428.0,-0.293186,0.414799
856,3702,大聯大,電子–電子通路,3.97,4.02,3.93,12.82,57306638.0,-0.012438,0.012594
1183,5522,遠雄,建材營建,32.00,13.36,29.05,10.87,2235271.0,1.395210,-0.582500
1426,6505,台塑化,油電燃氣,7.83,-8.11,-11.92,-46.49,33532841.0,-1.965475,-2.035760
1525,8044,網家,電子商務,10.80,11.91,11.62,16.79,3447881.0,-0.093199,0.102778


In [41]:
data.groupby(['2020產業名稱']).get_group('水泥工業')

,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
0,1101,台泥,水泥工業,29.47,26.30,35.71,-2.67,10178622.0,0.120532,-0.107567
1,1102,亞泥,水泥工業,28.07,20.53,32.27,-12.83,6798878.0,0.367267,-0.268614
2,1103,嘉泥,水泥工業,7.37,-3.93,-0.62,17.18,173129.0,-2.875318,-1.533243
3,1104,環泥,水泥工業,15.20,13.44,16.84,13.66,477040.0,0.130952,-0.115789
4,1108,幸福,水泥工業,10.52,16.55,18.75,23.73,384035.0,-0.364350,0.573194
5,1109,信大,水泥工業,34.35,27.72,36.18,-3.99,656183.0,0.239177,-0.193013
6,1110,東泥,水泥工業,6.94,2.28,3.04,0.07,124643.0,2.043860,-0.671470


In [42]:
#選出月營收成長最高的股票
df4=data.groupby(['2020產業名稱'])[three_months_income].transform(max) == data[three_months_income]
bool_filter2 = data[df4]['2020產業名稱'].isin(list(df_good_profit.index))
result2=data[df4][bool_filter2]
result2

,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
49,1325,恆大,塑膠工業,14.07,43.10,68.80,502.38,399921.0,-0.673550,2.063255
73,1438,裕豐,建材營建,0.00,63.00,73.76,691900.00,0.0,-1.000000,NaN
228,1906,寶隆,造紙工業,9.68,10.27,10.21,22.37,148465.0,-0.057449,0.060950
612,3055,蔚華科,電子–電子通路,15.61,13.80,18.86,74.66,204298.0,0.131159,-0.115951
849,3687,歐買尬,電子商務,35.40,37.01,37.63,21.48,111607.0,-0.043502,0.045480
910,4171,瑞基,農業科技,70.36,75.12,76.69,226.83,52013.0,-0.063365,0.067652
1220,6020,大展證,金融保險,96.37,104.93,97.76,847.86,115247.0,-0.081578,0.088824
1248,6130,星寶國際,生技醫療,67.54,75.10,63.41,412.20,30801.0,-0.100666,0.111934
1462,6582,申豐,橡膠工業,27.61,27.81,46.24,56.53,416832.0,-0.007192,0.007244
1661,8926,台汽電,油電燃氣,6.58,5.17,6.66,70.00,774384.0,0.272727,-0.214286


In [43]:
#選出月營收成長前三高的股票
df6=data.sort_values(three_months_income, ascending=False)
df7=pd.DataFrame(columns=Columns)
for i in list(df_good_profit.index):
    df7=pd.concat([df7,df6[data["2020產業名稱"]==i].head(3)],axis=0)
df7



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
910,4171,瑞基,農業科技,70.36,75.12,76.69,226.83,52013.0,-0.063365,0.067652
26,1240,茂生農經,農業科技,12.72,14.68,16.76,22.86,180860.0,-0.133515,0.154088
1459,6578,達邦蛋白,農業科技,14.03,12.25,17.58,-15.62,57797.0,0.145306,-0.126871
228,1906,寶隆,造紙工業,9.68,10.27,10.21,22.37,148465.0,-0.057449,0.060950
226,1904,正隆,造紙工業,25.02,28.85,25.65,-0.96,3364103.0,-0.132756,0.153078
225,1903,士紙,造紙工業,33.88,26.29,31.42,-7.42,11790.0,0.288703,-0.224026
49,1325,恆大,塑膠工業,14.07,43.10,68.80,502.38,399921.0,-0.673550,2.063255
46,1321,大洋,塑膠工業,3.18,5.01,13.16,0.52,450073.0,-0.365269,0.575472
1598,8354,冠好,塑膠工業,15.31,21.25,23.05,-1.31,74440.0,-0.279529,0.387982
849,3687,歐買尬,電子商務,35.40,37.01,37.63,21.48,111607.0,-0.043502,0.045480


In [44]:
profit_threshold1=result1[profit_rate]> profit_filter
profit_threshold2=result2[profit_rate]> profit_filter
#第一組
print('第一組:')
print('整個板塊毛利中位數成長(QoQ)靠前，而且單月營收在板塊當中最高，而且毛利率大於整體股市的中位數:')
result1[profit_threshold1]

第一組:
整個板塊毛利中位數成長(QoQ)靠前，而且單月營收在板塊當中最高，而且毛利率大於整體股市的中位數:


,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
197,1736,喬山,生技醫療,48.60,47.13,46.76,9.96,2172534.0,0.031190,-0.030247
272,2105,正新,橡膠工業,21.24,18.10,23.25,-13.23,8988799.0,0.173481,-0.147834
1183,5522,遠雄,建材營建,32.00,13.36,29.05,10.87,2235271.0,1.395210,-0.582500


In [45]:
#第二組
print('第二組:')
print('整個板塊毛利中位數成長(QoQ)靠前，而且近三月合併營收年成長在板塊當中最高，而且毛利率大於整體股市的中位數:')
result2[profit_threshold2]

第二組:
整個板塊毛利中位數成長(QoQ)靠前，而且近三月合併營收年成長在板塊當中最高，而且毛利率大於整體股市的中位數:


,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
49,1325,恆大,塑膠工業,14.07,43.10,68.80,502.38,399921.0,-0.673550,2.063255
73,1438,裕豐,建材營建,0.00,63.00,73.76,691900.00,0.0,-1.000000,NaN
849,3687,歐買尬,電子商務,35.40,37.01,37.63,21.48,111607.0,-0.043502,0.045480
910,4171,瑞基,農業科技,70.36,75.12,76.69,226.83,52013.0,-0.063365,0.067652
1220,6020,大展證,金融保險,96.37,104.93,97.76,847.86,115247.0,-0.081578,0.088824
1248,6130,星寶國際,生技醫療,67.54,75.10,63.41,412.20,30801.0,-0.100666,0.111934
1462,6582,申豐,橡膠工業,27.61,27.81,46.24,56.53,416832.0,-0.007192,0.007244


In [46]:
#第三組，跟第一組一樣，只是本來一個板塊選一檔，變成選三檔。
profit_threshold3=df5[profit_rate]> profit_filter

print('第三組:')
print('整個板塊毛利中位數成長(QoQ)靠前，而且單月營收在板塊當中是前三名，而且毛利率大於整體股市的中位數:')
df5[profit_threshold3]

第三組:
整個板塊毛利中位數成長(QoQ)靠前，而且單月營收在板塊當中是前三名，而且毛利率大於整體股市的中位數:


,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
1428,6508,惠光,農業科技,23.97,33.57,30.42,-17.50,156184.0,-0.285970,0.400501
226,1904,正隆,造紙工業,25.02,28.85,25.65,-0.96,3364103.0,-0.132756,0.153078
1107,5287,數字,電子商務,75.99,79.28,79.17,5.18,135277.0,-0.041498,0.043295
1183,5522,遠雄,建材營建,32.00,13.36,29.05,10.87,2235271.0,1.395210,-0.582500
443,2520,冠德,建材營建,26.05,25.78,24.87,64.87,1635701.0,0.010473,-0.010365
272,2105,正新,橡膠工業,21.24,18.10,23.25,-13.23,8988799.0,0.173481,-0.147834
273,2106,建大,橡膠工業,21.43,18.93,23.41,-7.02,2922815.0,0.132066,-0.116659
197,1736,喬山,生技醫療,48.60,47.13,46.76,9.96,2172534.0,0.031190,-0.030247
213,1795,美時,生技醫療,37.53,45.89,40.23,41.89,1116873.0,-0.182175,0.222755
156,1598,岱宇,生技醫療,30.66,28.04,45.13,175.87,1042445.0,0.093438,-0.085453


In [47]:
#第四組，同第二組，只是多選
profit_threshold4=df7[profit_rate]> profit_filter

print('第四組:')
print('整個板塊毛利中位數成長(QoQ)靠前，而且近三月合併營收年成長在板塊當中前三名，而且毛利率大於整體股市的中位數:')
df7[profit_threshold4]

第四組:
整個板塊毛利中位數成長(QoQ)靠前，而且近三月合併營收年成長在板塊當中前三名，而且毛利率大於整體股市的中位數:


,股票代號,股票名稱,2020產業名稱,2019Q4毛利率(%),2020Q1毛利率(%),2020Q2毛利率(%),202007近三月合併營收年成長(%),202007單月合併營收(千),自訂.季毛利率成長率,自訂.前兩季的毛利率QoQ成長率
910,4171,瑞基,農業科技,70.36,75.12,76.69,226.83,52013.0,-0.063365,0.067652
226,1904,正隆,造紙工業,25.02,28.85,25.65,-0.96,3364103.0,-0.132756,0.153078
225,1903,士紙,造紙工業,33.88,26.29,31.42,-7.42,11790.0,0.288703,-0.224026
49,1325,恆大,塑膠工業,14.07,43.10,68.80,502.38,399921.0,-0.673550,2.063255
1598,8354,冠好,塑膠工業,15.31,21.25,23.05,-1.31,74440.0,-0.279529,0.387982
849,3687,歐買尬,電子商務,35.40,37.01,37.63,21.48,111607.0,-0.043502,0.045480
1107,5287,數字,電子商務,75.99,79.28,79.17,5.18,135277.0,-0.041498,0.043295
1657,8917,欣泰,油電燃氣,27.68,20.41,25.92,-2.36,175825.0,0.356198,-0.262645
73,1438,裕豐,建材營建,0.00,63.00,73.76,691900.00,0.0,-1.000000,NaN
461,2596,綠意,建材營建,34.15,28.41,29.02,15574.15,27496.0,0.202042,-0.168082
